<a href="https://colab.research.google.com/github/OmarKhaledAbdlhafez/RSNA_classification/blob/master/RSNA_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/rsna

/content/gdrive/My Drive/rsna


In [0]:
!ls

 data.pickle				  stage_2_test_images
'GCP Credits Request Link - RSNA.txt'	  stage_2_train_images
 rsna-pneumonia-detection-challenge.zip   stage_2_train_labels.csv
 stage_2_detailed_class_info.csv	  test
 stage_2_sample_submission.csv


In [0]:
import os 
print(len(os.listdir('stage_2_train_images')))

OSError: ignored

In [0]:
!mkdir test 

In [0]:
import shutil
i=0
src='stage_2_train_images/'
for file in os.listdir('stage_2_train_images'):
  shutil.move(src+file, "test/")
  i=i+1
  if i==6000:
    break


In [0]:
import pandas as pd 
df = pd.read_csv('stage_2_train_labels.csv')
df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [0]:
sick= df[df.Target==1]
sick= sick[['patientId','Target']]
sick.head()

,patientId,Target
4,00436515-870c-4b36-a041-de91049b9ab4,1
5,00436515-870c-4b36-a041-de91049b9ab4,1
8,00704310-78a8-4b38-8475-49f4573b2dbb,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,1
14,00aecb01-a116-45a2-956c-08d2fa55433f,1


In [0]:
sickfiles=[]
for idx , row  in sick.iterrows():
   name = row['patientId']
   sickfiles.append(name)


In [0]:
print(len(sickfiles))

9555


In [0]:
!pip install pydicom

In [0]:
from keras.utils import Sequence
import pydicom, numpy as np
from skimage.transform import resize
class generator(Sequence):
    
    def __init__(self, folder, filenames,sickfiles,batch_size=32, image_size=256, shuffle=True):
        self.folder = folder
        self.filenames = filenames
        self.sickfiles=sickfiles
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in self.sickfiles:
            y=1
        else :
          y=0    
        img = resize(img, (256, 256), anti_aliasing=True)
        img = np.expand_dims(img, -1)
        return img, y
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        items = [self.__load__(filename) for filename in filenames]
        imgs, y = zip(*items)
        imgs = np.array(imgs)
        y = np.array(y)
        return imgs, y
         
    def on_epoch_end(self):
        if self.shuffle:
           np.random.shuffle(self.filenames)
        
    def __len__(self):
      return int(len(self.filenames) / self.batch_size)

In [0]:
folder = 'stage_2_train_images/'
filenames = os.listdir(folder)
np.random.shuffle(filenames)
# split into train and validation filenames
n_valid_samples = 4000
n_train_samples = len(filenames) - n_valid_samples
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))

n train samples 16684
n valid samples 4000


In [0]:
train_gen = generator(folder, train_filenames,
                      sickfiles, batch_size=32,
                      image_size=256, shuffle=True)
valid_gen = generator(folder, valid_filenames, 
                      sickfiles, batch_size=32, 
                      image_size=256, shuffle=False)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 256)      

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit_generator(train_gen,validation_data=valid_gen,epochs=10)

Epoch 1/10
521/521 [==============================] - 4419s 8s/step - loss: 0.4620 - accuracy: 0.7790 - val_loss: 0.4512 - val_accuracy: 0.7947
Epoch 2/10
521/521 [==============================] - 1406s 3s/step - loss: 0.4465 - accuracy: 0.7913 - val_loss: 0.4478 - val_accuracy: 0.7962
Epoch 3/10
521/521 [==============================] - 1389s 3s/step - loss: 0.4364 - accuracy: 0.7953 - val_loss: 0.4392 - val_accuracy: 0.7980
Epoch 4/10
521/521 [==============================] - 1416s 3s/step - loss: 0.4233 - accuracy: 0.8034 - val_loss: 0.4521 - val_accuracy: 0.7943
Epoch 5/10
520/521 [============================>.] - ETA: 2s - loss: 0.4147 - accuracy: 0.8079